## Multi Label(Head, Branch) Classifier
- 하나의 Convolution 모델에서 3개의 FC Layer 브랜치를 만들어보자

In [1]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [2]:
# Custom Model Template
class Res50(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """
        self.pretrain_model = torchvision.models.resnext50_32x4d(pretrained=True)
        self.pretrain_model.fc = torch.nn.Linear(in_features=2048, out_features=num_classes, bias=True) # resnet18.fc의 in_features의 크기는?
        # torch.nn.init.xavier_uniform_(pretrain_model.fc.weight)
        # stdv = 1.0/np.sqrt(512) # fully connected layer의 bias를 resnet18.fc in_feature의 크기의 1/root(n) 크기의 uniform 분산 값 중 하나로 설정해주세요! - Why? https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch
        # pretrain_model.fc.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.pretrain_model.forward(x)
        return x


# Custom Model Template
class MultiRes50(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """
        self.res50 = Res50(num_classes)
        self.res50.load_state_dict(torch.load("/opt/ml/workspace/code/model/res50CusDS3/best.pth"))
        self.res50.to("cpu")
        
        self.res50 = nn.Sequential(*list(self.res50.pretrain_model.children())[:-1])

        self.mask = nn.Linear(2048, 3, bias=True)
        self.age = nn.Linear(2048, 3, bias=True)
        self.gender = nn.Linear(2048, 3, bias=True)
        # torch.nn.init.xavier_uniform_(pretrain_model.fc.weight)
        # stdv = 1.0/np.sqrt(512) # fully connected layer의 bias를 resnet18.fc in_feature의 크기의 1/root(n) 크기의 uniform 분산 값 중 하나로 설정해주세요! - Why? https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch
        # pretrain_model.fc.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.res50.forward(x)
        x = torch.flatten(x, start_dim=1)
        m = self.mask(x)
        a = self.age(x)
        s = self.gender(x)
        return {"mask":m, "age":a, "gender":s}

In [3]:
def dfs_freeze(model):
    for name, child in model.named_children():
        for param in child.parameters():
            #print(param)
            param.requires_grad = False
            #print(param)
        dfs_freeze(child)

In [4]:
mask_model = MultiRes50(18)

In [5]:
dfs_freeze(mask_model.res50)

In [6]:
# from torchsummary import summary
# summary(mask_model.to(device),input_size=(3,320,320))

- 기존 train data 1가지
- mask, gender, age 각각 labeling을 한 데이터프레임 3가지
- 총 4가지의 데이터프레임을 Dataset모듈에 넣을 것임

In [7]:
# # 기본 train 데이터 셋
df = pd.read_csv('train_label.csv')
# df.head(5)

In [8]:
def split_data(df):
    # # wear, incorrect, normal 3가지 클래스로 변경
    # # 0~5, 6~11, 12~17끼리 묶는다
    df_mask = df.copy()

    def mask_label(x):
        if x in [0,1,2,3,4,5]:
            return 0    # wear
        elif x in [6,7,8,9,10,11]:
            return 1    # incorrect
        else:
            return 2    # not wear

    df_mask['label'] = df_mask['label'].apply(mask_label)

    # # 남성, 여성 2가지 클래스로 변경
    # # [0,1,2,6,7,8,12,13,14], [3,4,5,9,10,11,15,16,17]
    df_gender = df.copy()

    def gender_label(x):
        if x in [0,1,2,6,7,8,12,13,14]:
            return 0    # male
        else:
            return 1    # female

    df_gender['label'] = df_gender['label'].apply(gender_label)

    # # 청년, 중년, 장년 3가지 클래스로 변경
    # # [0,3,6,9,12,15], [1,4,7,10,13,16], [2,5,8,11,14,17]
    df_age = df.copy()

    def age_label(x):
        if x in [0,3,6,9,12,15]:
            return 0    # young
        elif x in [1,4,7,10,13,16]:
            return 1    # middle
        else:
            return 2    # old

    df_age['label'] = df_age['label'].apply(age_label)

    return df_mask['label'],df_age['label'],df_gender['label']


### Dataset  정의
- mask, gender, age 별로 label 나눔

In [9]:
class MultiBranchDataset(Dataset):
    def __init__(self, df, transforms):
        self.df = df
        self.image_data = self.df['path']   # x data, 이미지
        self.image_label = self.df['label'] # y data, 레이블


        self.mask_label, self.age_label, self.gender_label = split_data(df)
        self.transform = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        img_path = self.df['path'].iloc[idx]
        img = Image.open(img_path)

        if self.transform:
            img = self.transform(img)
        
        dict_label = {
            'class' : self.image_label.iloc[idx],
            'mask' : self.mask_label.iloc[idx],
            'gender' : self.gender_label.iloc[idx],
            'age' : self.age_label.iloc[idx]
        }

        return img, dict_label

class CusDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.image_data = self.df['path']   # x data, 이미지
        self.image_label = self.df['label'] # y data, 레이블

        self.transform = transform

    def __getitem__(self, idx):
        image = Image.open(self.image_data.iloc[idx])
        label = self.image_label.iloc[idx]

        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)

    def __len__(self):
        return len(self.image_label)


#### Transform & Dataset
- 모든 데이터를 한방에 학습할 때와
- train과 valid를 split할 때의 코드가 다름 (train, valid를 쪼갠거와 동일한 mask, gender, age 레이블링이 필요함)

In [10]:
# Transform Compose
data_transform = torchvision.transforms.Compose([
    # transforms.CenterCrop(320),
    torchvision.transforms.CenterCrop(320),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.2,0.2,0.2)),
])


from sklearn.model_selection import train_test_split
train, valid = train_test_split(df, test_size = 0.25, shuffle=True, stratify=df['label'], random_state=1234)
print(train.shape, valid.shape)

# train_dataset = CusDataset(train, data_transform)
# test_dataset = CusDataset(valid, data_transform)

# train data 한방에 학습시키는 경우
train_dataset = MultiBranchDataset(train,data_transform)
test_dataset = MultiBranchDataset(valid,data_transform)

(14175, 3) (4725, 3)


### DataLoader
- 여기서 3개로 나눌 필요가 없음
- 모델 train에서 loss를 따로 나눌 것임

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=50, 
        num_workers=2,
        shuffle=True,
        pin_memory=torch.cuda.is_available(),
        drop_last=True,)

test_dataloader = DataLoader(test_dataset, batch_size=50, 
        num_workers=2,
        shuffle=True,
        pin_memory=torch.cuda.is_available(),
        drop_last=True,)

In [12]:
dataloaders = {
    "train" : train_dataloader,
    "test" : test_dataloader
}

데이터로더의 label 부분을 보면 mask, gender, age별로 배치사이즈에 맞게 레이블 정보가 담아짐

In [13]:
next(iter(train_dataloader))[1]

{'class': tensor([ 3,  3,  1, 10,  3,  2, 12,  3,  0,  3,  4,  3,  5,  4, 10,  3,  1, 16,
          3, 15, 12, 12,  0,  0,  4,  3, 16,  3, 12,  3,  4,  3,  3,  4, 12,  0,
         13,  3,  0,  7,  9, 10,  1,  3, 11,  3,  4,  3,  3,  3]),
 'mask': tensor([0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 2, 2, 2, 0, 0,
         0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
         0, 0]),
 'gender': tensor([1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
         1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
         1, 1]),
 'age': tensor([0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
         1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 2, 0, 1, 0,
         0, 0])}

#### Conv layer 빠져나오고 브랜치 만들기

In [14]:
# class MultiBranchModel(nn.Module):
#     def __init__(self):
#         super().__init__()
        
#         self.base_model = torchvision.models.resnet18(pretrained=True)

#         self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])

#         self.mask = nn.Sequential(
#             # nn.Dropout(p=0.2),
#             nn.Linear(512, 3, bias=True)
#         )
#         self.gender = nn.Sequential(
#             # nn.Dropout(p=0.2),
#             nn.Linear(512, 2, bias=True)
#         )
#         self.age = nn.Sequential(
#             # nn.Dropout(p=0.2),
#             nn.Linear(512, 3, bias=True)
#         )
        

#     def forward(self, x):
#         x = self.base_model(x)
#         # print(x.shape)

#         x = torch.flatten(x, start_dim=1)
#         # print('22: ', x.shape)

#         return {
#             'mask': self.mask(x),
#             'gender': self.gender(x),
#             'age': self.age(x)
#         }

#     # Loss 함수 구현 부분
#     def get_loss(self, net_output, ground_truth):
#         mask_loss = F.cross_entropy(net_output['mask'], ground_truth['mask'].to(device))
#         gender_loss = F.cross_entropy(net_output['gender'], ground_truth['gender'].to(device))
#         age_loss = F.cross_entropy(net_output['age'], ground_truth['age'].to(device))

#         loss = mask_loss + gender_loss + age_loss

#         return loss, {'mask' : mask_loss, 'gender' : gender_loss, 'age' : age_loss}

### Loss 함수 정의
- 3가지 task 각각 loss 구하고 합침

In [15]:
# def get_loss(net_output, ground_truth):
#     mask_loss = F.cross_entropy(net_output['mask'], ground_truth['mask'])
#     gender_loss = F.cross_entropy(net_output['gender'], ground_truth['gender'])
#     age_loss = F.cross_entropy(net_output['age'], ground_truth['age'])

#     loss = mask_loss + gender_loss + age_loss

#     return loss, {'mask' : mask_loss, 'gender' : gender_loss, 'age' : age_loss}

## Train

In [31]:
target_model = mask_model.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 3

optimizer = {
        "mask":torch.optim.Adam(mask_model.mask.parameters(), lr=LEARNING_RATE),
        "age":torch.optim.Adam(mask_model.age.parameters(), lr=LEARNING_RATE),
        "gender":torch.optim.Adam(mask_model.gender.parameters(), lr=LEARNING_RATE)
    }
#optimizer = torch.optim.Adam(mask_model.parameters(), lr=LEARNING_RATE)

mask_loss_fn = torch.nn.CrossEntropyLoss()
age_loss_fn = torch.nn.CrossEntropyLoss()
gender_loss_fn = torch.nn.CrossEntropyLoss()
loss_fn = {"mask":mask_loss_fn,"age":age_loss_fn,"gender":gender_loss_fn}

In [17]:
from tqdm.notebook import tqdm 

In [34]:
best_test_accuracy = {"mask":0.,"age":0.,"gender":0.}
best_test_loss = {"mask":9999.,"age":9999.,"gender":9999.}

for epoch in range(NUM_EPOCH):
    for phase in ["train", "test"]:
        running_loss = {"mask":0.,"age":0.,"gender":0.}
        running_acc = {"mask":0.,"age":0.,"gender":0.}
        if phase == "train":
            target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
        elif phase == "test":
            target_model.eval()

        for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
            images = images.to(device)
            labels = labels#.to(device) 

            for k in ["mask","age","gender"]:
                optimizer[k].zero_grad()
            loss = {}
            preds = {}
            with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
                logits = target_model(images)

                for k,v in logits.items():
                    _, preds[k] = torch.max(v, 1)
                    loss[k] = loss_fn[k](v, labels[k].to(device))

                if phase == "train":
                    for k,v in loss.items():
                        loss[k].backward()
                        optimizer[k].step() # 계산된 gradient를 가지고 모델 업데이트

            for k,v in loss.items():
                running_loss[k] += loss[k].item() * images.size(0) # 한 Batch에서의 loss 값 저장
                running_acc[k] += torch.sum(preds[k] == labels[k].data.to(device)) # 한 Batch에서의 Accuracy 값 저장

        epoch_loss = {}
        epoch_acc = {}
        # 한 epoch이 모두 종료되었을 때,
        for k in ["mask","age","gender"]:
            epoch_loss[k] = running_loss[k] / len(dataloaders[phase].dataset)
            epoch_acc[k] = running_acc[k] / len(dataloaders[phase].dataset)
            print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss[k]:.3f}, 평균 Accuracy : {epoch_acc[k]:.3f}")
            if phase == "test" and best_test_accuracy[k] < epoch_acc[k]: # phase가 test일 때, best accuracy 계산
                best_test_accuracy[k] = epoch_acc[k]
            if phase == "test" and best_test_loss[k] > epoch_loss[k]: # phase가 test일 때, best loss 계산
                best_test_loss[k] = epoch_loss[k]
        # if i % 50 == 0:
        #     print('Epoch: {}, i: {},Loss: {:.6f}'.format(epoch, i, running_loss))
        #     print(f'{i}번 배치: {running_acc}/{(i+1)*64}, 정확도: {running_acc/((i+1)*64)}')
for k in ["mask","age","gender"]: 
    print("학습 종료!")
    print(f"최고 accuracy : {best_test_accuracy[k]}, 최고 낮은 loss : {best_test_loss[k]}")



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.004, 평균 Accuracy : 0.997
현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.058, 평균 Accuracy : 0.982
현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.018, 평균 Accuracy : 0.994



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.001, 평균 Accuracy : 0.994
현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.047, 평균 Accuracy : 0.981
현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.015, 평균 Accuracy : 0.991



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.006, 평균 Accuracy : 0.997
현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.059, 평균 Accuracy : 0.981
현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.018, 평균 Accuracy : 0.994



현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.001, 평균 Accuracy : 0.994
현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.044, 평균 Accuracy : 0.983
현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.014, 평균 Accuracy : 0.992



현재 epoch-2의 train-데이터 셋에서 평균 Loss : 0.004, 평균 Accuracy : 0.997
현재 epoch-2의 train-데이터 셋에서 평균 Loss : 0.058, 평균 Accuracy : 0.982
현재 epoch-2의 train-데이터 셋에서 평균 Loss : 0.018, 평균 Accuracy : 0.994



현재 epoch-2의 test-데이터 셋에서 평균 Loss : 0.001, 평균 Accuracy : 0.994
현재 epoch-2의 test-데이터 셋에서 평균 Loss : 0.043, 평균 Accuracy : 0.983
현재 epoch-2의 test-데이터 셋에서 평균 Loss : 0.015, 평균 Accuracy : 0.991
학습 종료!
최고 accuracy : 0.9944973587989807, 최고 낮은 loss : 0.000657279985067102
학습 종료!
최고 accuracy : 0.9832804799079895, 최고 낮은 loss : 0.043468732179866895
학습 종료!
최고 accuracy : 0.991534411907196, 최고 낮은 loss : 0.014447200169636519


### 테스트(Evaluation) 중, mask, gender, age별 output 뽑아내기
- 최종 클래스 예측(18개)으로 변환
- mask : 0, 1, 2 (wear, incorrect, not wear)
- gender : 0, 1  (male, female)
- age : 0, 1, 2  (young, middle, old)
- class : (mask * 6) + (gender * 3) + (age)

In [ ]:
output = mask_model(image)
pred_mask = torch.argmax(output['mask'], dim=-1)
pred_gender = torch.argmax(output['gender'], dim=-1)
pred_age = torch.argmax(output['age'], dim=-1)

pred_class = (pred_mask * 6) + (pred_gender * 3) + (pred_age)

## Testing

In [35]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)


        return image

    def __len__(self):
        return len(self.img_paths)

In [74]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

dataset = TestDataset(image_paths, data_transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
test_model = target_model.to(device)
test_model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        output = test_model(images)

        pred_mask = torch.argmax(output['mask'], dim=-1)
        pred_gender = torch.argmax(output['gender'], dim=-1)
        pred_age = torch.argmax(output['age'], dim=-1)
        pred_class = (pred_mask * 6) + (pred_gender * 3) + (pred_age)

        all_predictions.extend(pred_class.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission3ways2.csv'), index=False)
print('test inference is done!')


test inference is done!


### Save

In [38]:
torch.save(target_model.mask.state_dict(), "./resnext50_32x4dfc3ways_mask.pt")

### Performance

In [91]:
train_dir = '/opt/ml/input/data/train'
valid = pd.read_csv(os.path.join(train_dir, 'train_label.csv'))


In [58]:
class CusDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.image_data = self.df['path']   # x data, 이미지
        self.image_label = self.df['label'] # y data, 레이블

        self.transform = transform

    def __getitem__(self, idx):
        image = Image.open(self.image_data.iloc[idx])
        label = self.image_label.iloc[idx]

        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)

    def __len__(self):
        return len(self.image_label)


In [88]:
def check_eval(raw_data, dataloader, model, device):
    all_predictions = []
    with torch.no_grad():
        for i , (X,y) in enumerate(tqdm(dataloader)):
            model_pred = model.forward(X.to(device))

            pred_mask = torch.argmax(model_pred['mask'], dim=-1)
            pred_gender = torch.argmax(model_pred['gender'], dim=-1)
            pred_age = torch.argmax(model_pred['age'], dim=-1)
            pred_class = (pred_mask * 6) + (pred_gender * 3) + (pred_age)

            all_predictions.extend([[valid.iloc[i]['path'], pred_class.cpu().numpy()[0],y.cpu().numpy()[0]]])
    #print(all_predictions)
    result = pd.DataFrame(all_predictions, columns=['path', 'pred', 'target'])
    return result

In [ ]:
valid_dataset = CusDataset(valid, data_transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=1, 
        num_workers=8,
        shuffle=True,
        pin_memory=torch.cuda.is_available(),
        drop_last=True,
    )

check_eval_df = check_eval(valid, valid_dataloader, target_model, device)
check_eval_df

In [93]:
wrong_df = check_eval_df[check_eval_df['pred'] != check_eval_df['target']]
wrong_df = wrong_df.reset_index(drop=True)
# wrong_df.head()
print(len(wrong_df))

305


In [94]:
from sklearn.metrics import f1_score
f1_score(check_eval_df['target'], check_eval_df['pred'], average='macro')

0.9670378591343353